In [ ]:
import torch
import cv2
import os
import json
import re
import numpy as np

class ObjectDetector:
    def __init__(self, model_name='yolov5l', confidence_threshold=0.5):
        self.model = torch.hub.load('ultralytics/yolov5', model_name, pretrained=True)
        self.model.eval()
        self.confidence_threshold = confidence_threshold
        self.excluded_objects = self.load_excluded_objects("image1.txt")

    def detect_objects(self, image_path):
        if not os.path.isfile(image_path):
            raise FileNotFoundError(f"Input image file {image_path} does not exist or is not readable")

        image = cv2.imread(image_path)

        yolov5_results = self.model(image)

        objects = []
        for label, box, conf in zip(yolov5_results.xyxy[0][:, -1], yolov5_results.xyxy[0][:, :4], yolov5_results.xyxy[0][:, 4]):
            if conf >= self.confidence_threshold:
                class_label = self.extract_object_label(self.model.names[int(label)])
                if class_label not in self.excluded_objects:
                    box = box.detach().cpu().numpy()
                    objects.append({
                        'object': class_label,
                        'confidence': conf.item(),
                        'bbox': [round(i, 2) for i in box.tolist()]
                    })

        return objects

    def load_excluded_objects(self, txt_file):
        excluded_objects = []
        with open(txt_file, 'r') as file:
            for line in file:
                excluded_objects.append(line.strip())
        return excluded_objects

    def extract_object_label(self, label):
        matches = re.search(r'\{(.*?)\}', label)
        if matches:
            return matches.group(1)
        else:
            return label

    def draw_boxes_on_image(self, image, detected_objects):
        for obj in detected_objects:
            class_label = obj['object']
            bbox = obj['bbox']
            x1, y1, x2, y2 = bbox
            x1, y1, x2, y2 = max(0, int(x1)), max(0, int(y1)), min(image.shape[1], int(x2)), min(image.shape[0], int(y2))
            
            if class_label == "cartoon milk":
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image, "Cartoon Milk", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            else:
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(image, class_label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    def process_images_in_folder(self, input_folder, output_folder, output_json_file):
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        detected_objects_list = []

        for filename in os.listdir(input_folder):
            if filename.endswith(('.jpg', '.png', '.jpeg', '.bmp')):
                image_path = os.path.join(input_folder, filename)
                detected_objects = self.detect_objects(image_path)
                detected_objects_list.extend(detected_objects)

                image = cv2.imread(image_path)
                self.draw_boxes_on_image(image, detected_objects)
                output_path = os.path.join(output_folder, filename)
                cv2.imwrite(output_path, image)

        with open(output_json_file, 'w') as json_file:
            objects_dict_list = [{'object': obj['object'], 'confidence': obj['confidence'], 'bbox': obj['bbox']} for obj in detected_objects_list]
            json.dump(objects_dict_list, json_file, indent=4)

if __name__ == "__main__":
    detector = ObjectDetector()

    input_folder = "F:\\dairies\\dairies"
    output_folder = "output_images_10_folder"
    output_json_file = "detected_objects.json"

    try:
        detector.process_images_in_folder(input_folder, output_folder, output_json_file)
        print(f"Object detection completed. Images with bounding boxes saved in {output_folder}")
        print(f"Detected objects saved in {output_json_file}")
    except Exception as e:
        print(f"Error: {str(e)}")
